# Correlation for a single GRB

Generates the data to be analyzed

## Import all necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import popsynth
from popsynth.aux_samplers.trunc_normal_aux_sampler import TruncatedNormalAuxSampler
from popsynth.aux_samplers.lognormal_aux_sampler import LogNormalAuxSampler
from popsynth.aux_samplers.delta_aux_sampler import DeltaAuxSampler

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from zusammen.synthetic_populations.aux_samplers import TDecaySampler, DurationSampler, LumSampler, EpeakObsSampler

%matplotlib widget
import cosmogrb

## Simulate one GRB in popsynth

In [ ]:
pop_gen = popsynth.populations.SFRPopulation(r0=1/1.5, rise=1., decay=4.0,
                                             peak=1.5, r_max=7.0, a=0.1)

t90 = LogNormalAuxSampler(name="t90", observed=False)
t90.mu=np.log10(10)
t90.tau=0.25

ep = LogNormalAuxSampler(name="ep", observed=False)
ep.mu= np.log(300)
ep.tau=0.4

alpha = TruncatedNormalAuxSampler(name="alpha", observed=False)
alpha.lower=-1.5
alpha.upper=0.0
alpha.mu=-1
alpha.tau=0.25

ep_tau = TruncatedNormalAuxSampler(name="ep_tau", observed=False)
ep_tau.lower=-2
ep_tau.upper=-1
ep_tau.mu=-1.5
ep_tau.tau=0.25

# nrest = LogNormalAuxSampler(name="nrest", observed=False)
# nrest.mu = np.log(1e52)
# nrest.tau = 0.1
nrest = DeltaAuxSampler(name="nrest", observed=False)
nrest.xp = 1e52

# gamma = TruncatedNormalAuxSampler(name="gamma", observed=False)
# gamma.mu = 1.5
# gamma.tau = 0.5
# gamma.lower = 1.0
# gamma.upper = 2.0
gamma = DeltaAuxSampler(name="gamma", observed=False)
gamma.xp = 1.5

duration = DurationSampler()
obs_lum = LumSampler()
ep_obs = EpeakObsSampler()

duration.set_secondary_sampler(t90)
obs_lum.set_secondary_sampler(ep)
obs_lum.set_secondary_sampler(nrest)
obs_lum.set_secondary_sampler(gamma)
ep_obs.set_secondary_sampler(ep)

pop_gen.add_observed_quantity(duration)
pop_gen.add_observed_quantity(obs_lum)
pop_gen.add_observed_quantity(ep_obs)
pop_gen.add_observed_quantity(alpha)
pop_gen.add_observed_quantity(ep_tau)

In [ ]:
population = pop_gen.draw_survey()
population.writeto('data/single_grb.h5')

In [ ]:
population

## Define GRB

In [ ]:
from corr_cpl.corr_cpl_source import CorrCPLSourceFunction
from corr_cpl.corr_cpl_grb import GBMGRB_CORR_CPL

In [ ]:
grb = GBMGRB_CORR_CPL(
    ra=population.ra[0],
    dec=population.dec[0],
    z=population.distances[0],
    peak_flux=population.fluxes_latent[0],
    alpha=population.alpha[0],
    ep_start=population.ep_obs[0],
    ep_tau=population.ep_tau[0],
    Nrest=population.nrest[0],
    gamma=population.gamma[0],
    duration=population.duration[0],
    T0=0.,
)

grb.info()

Energy-integrated light curve:

In [ ]:
time = np.linspace(0, 20, 500)

grb.display_energy_integrated_light_curve(time)

Energy-dependent light curve:

In [ ]:
time = np.linspace(0, 20)
energy = np.logspace(1, 3, 100)

grb.display_energy_dependent_light_curve(time, energy, cmap='viridis')

Spectrum:

In [ ]:
time = np.linspace(0, 10, 10)
energy = np.logspace(1, 4, 100)
out = list(grb._lightcurves.values())[0]._source._source_function.evolution(energy, time)
fig, ax = plt.subplots()
for y, c in zip(out, plt.cm.viridis(np.linspace(0,1,len(out)))):
    ax.plot(energy, y, color=c)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-5, 1e1)
ax.set_xlabel('energy')
ax.set_ylabel('flux')

## Simulate GRB

In [ ]:
grb.go(serial=True)
grb.save('data/test_grb.h5')

In [ ]:
grb_reload = cosmogrb.GRBSave.from_file('data/test_grb.h5')

Light curves:

In [ ]:
fig, axes = plt.subplots(4,4,sharex=True,sharey=False,figsize=(10,10))
row=0
col = 0
for k,v  in grb_reload.items():
    ax = axes[row][col]

    lightcurve =v['lightcurve']

    lightcurve.display_lightcurve(dt=.5, ax=ax,lw=1,color='#25C68C')
    lightcurve.display_source(dt=.5,ax=ax,lw=1,color="#A363DE")
    lightcurve.display_background(dt=.5,ax=ax,lw=1, color="#2C342E")
    ax.set_xlim(-10, 30)
    ax.set_title(k,size=8)



    if col < 3:
        col+=1
    else:
        row+=1
        col=0

axes[3,2].set_visible(False)
axes[3,3].set_visible(False)
plt.tight_layout()

Spectra:

In [ ]:
fig, axes = plt.subplots(4,4,sharex=False,sharey=False,figsize=(10,10))
row=0
col = 0

for k, v in grb_reload.items():
    ax = axes[row][col]

    lightcurve = v['lightcurve']

    lightcurve.display_count_spectrum(tmin=0, tmax=5, ax=ax,color='#25C68C')
    lightcurve.display_count_spectrum_source(tmin=0, tmax=5, ax=ax,color="#A363DE")
    lightcurve.display_count_spectrum_background(tmin=0, tmax=5, ax=ax, color="#2C342E")
    ax.set_title(k,size=8)

    if col < 3:
        col+=1
    else:
        row+=1
        col=0

axes[3,2].set_visible(False)
axes[3,3].set_visible(False)
plt.tight_layout()

## Simulate universe

In [ ]:
from corr_cpl.corr_cpl_universe import GBM_CORR_CPL_Universe

In [ ]:
universe = GBM_CORR_CPL_Universe('data/single_grb.h5', save_path="data")

universe.go(client=None)

universe.save('data/survey.h5')

## View survey

In [ ]:
from cosmogrb.universe.survey import Survey

In [ ]:
survey = Survey.from_file('data/survey.h5')
survey.info()

In [ ]:
for i in range(len(survey)):
    print(f"GRB {i}")
    for j in ["alpha", "gamma", "Nrest"]:
        print(j, "=", survey['SynthGRB_' + str(i)].grb._source_params[j])
    print("z =", survey['SynthGRB_' + str(i)].grb.z)
    print()

Light curves:

In [ ]:
for grb_name in survey:
    fig, axes = plt.subplots(4,4,sharex=True,sharey=False,figsize=(10,10))
    row = 0
    col = 0
    for k,v in survey[grb_name].grb.items():
        ax = axes[row][col]

        lightcurve = v['lightcurve']

        lightcurve.display_lightcurve(dt=.5, ax=ax,lw=1,color='#25C68C')
        lightcurve.display_source(dt=.5,ax=ax,lw=1,color="#A363DE")
        lightcurve.display_background(dt=.5,ax=ax,lw=1, color="#2C342E")
        ax.set_xlim(-10, 30)
        ax.set_title(k,size=8)



        if col < 3:
            col+=1
        else:
            row+=1
            col=0

    #axes[3,2].set_visible(False)
    #axes[3,3].set_visible(False)

## Logging

In [ ]:
import logging
logger = logging.getLogger("cosmogrb")
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)